In [2]:
"""
Three-Agent Conversation (Son • Mother • Father)
Scenario: The son wants to buy a motorcycle. Dad is supportive; Mom is opposed.
This script alternates speakers and keeps a shared conversation history.

Requirements:
    pip install openai

How it works:
- You paste your API key below (manual method).
- Each agent has its own system prompt (role instruction).
- We keep a single shared conversation list; each turn we call the model
  with the active speaker's instructions + the conversation so far.
- We prefix every utterance with the speaker name (SON/MOTHER/FATHER) to keep it clear.
"""

from openai import OpenAI
import time

# -------------------------
# 1) Paste your API key here
# -------------------------
client = OpenAI(api_key="put-your-api-key-here")

# -------------------------
# 2) Model settings
# -------------------------
PRIMARY_MODEL = "gpt-4o-mini"   # fast & economical; change to "gpt-4o" if needed
FALLBACK_MODEL = "gpt-4o"
TEMPERATURE = 0.4               # lower = more focused/consistent

# -------------------------
# 3) Role instructions (system prompts)
# -------------------------
son_role = (
    "You are the SON. You want to buy a motorcycle. "
    "State your reasons respectfully (independence, budget, safety training), "
    "answer questions, and be open to conditions (classes, gear, savings). "
    "Keep replies to 1–4 sentences. Always start your message with 'SON: '."
)

mother_role = (
    "You are the MOTHER. You care about safety and finances and are currently against the purchase. "
    "Voice concerns calmly (injury risk, insurance, noise), ask clarifying questions, "
    "and suggest alternatives (postpone, safety course first, budget plan). "
    "Keep replies to 1–4 sentences. Always start your message with 'MOTHER: '."
)

father_role = (
    "You are the FATHER. You support the idea in principle, but only with strong safety rules and a clear budget. "
    "Help find a compromise (mandatory course, full gear, displacement limits, curfew, savings goals). "
    "Keep replies to 1–4 sentences. Always start your message with 'FATHER: '."
)

# -------------------------
# 4) Shared conversation history
#    We store messages and prefix who spoke in the content.
# -------------------------
conversation = [
    {"role": "system",
     "content": "This is a respectful family discussion. Keep it constructive, specific, and concise."}
]

def speak(role_instruction: str, conversation: list, model: str = PRIMARY_MODEL) -> str:
    """
    Ask the active agent to produce the next message based on the role instruction
    and the shared conversation so far. Falls back to a secondary model if needed.
    """
    try:
        resp = client.chat.completions.create(
            model=model,
            temperature=TEMPERATURE,
            messages=[{"role": "system", "content": role_instruction}] + conversation
        )
        return resp.choices[0].message.content.strip()
    except Exception as e:
        print(f"[WARN] {type(e).__name__}: {e} — retrying with fallback model '{FALLBACK_MODEL}'")
        time.sleep(0.5)
        resp = client.chat.completions.create(
            model=FALLBACK_MODEL,
            temperature=TEMPERATURE,
            messages=[{"role": "system", "content": role_instruction}] + conversation
        )
        return resp.choices[0].message.content.strip()

# -------------------------
# 5) Seed the conversation (Son starts)
# -------------------------
first_message = (
    "SON: I’d like to buy a small motorcycle. I’ve been researching safety courses "
    "and budgeting for gear, and I think I can handle it responsibly."
)
conversation.append({"role": "user", "content": first_message})
print(first_message)

# -------------------------
# 6) Turn order and loop
#    We'll do: Mother → Father → Son → Mother → Father → Son ... for N rounds
# -------------------------
turn_order = [
    ("MOTHER", mother_role),
    ("FATHER", father_role),
    ("SON", son_role),
]

ROUNDS = 3  # each round has 3 turns (Mother, Father, Son). Increase for longer chats.

for r in range(ROUNDS):
    for speaker_name, role_prompt in turn_order:
        reply = speak(role_prompt, conversation)
        # Ensure the speaker prefix is present; add if missing
        if not reply.upper().startswith(f"{speaker_name}:"):
            reply = f"{speaker_name}: " + reply

        # Print to console and append to history
        print(reply)
        # We can store all agent messages as 'assistant' for simplicity.
        # The key point is the speaker prefix inside the content.
        conversation.append({"role": "assistant", "content": reply})

# -------------------------
# 7) Optional wrap-up (Father proposes conditions or Mother summarizes concerns)
# -------------------------
wrap = speak(father_role, conversation)
if not wrap.upper().startswith("FATHER:"):
    wrap = "FATHER: " + wrap
print(wrap)
conversation.append({"role": "assistant", "content": wrap})

"""
Tips:
- Make the dialogue longer by increasing ROUNDS (e.g., 5 or 6).
- To bias toward a compromise, you can add a final prompt from MOTHER or SON.
- Adjust tone/length with TEMPERATURE and the role instructions (keep them brief and explicit).
- If you get a 'model not found' or quota error, switch models or check your billing.
"""


SON: I’d like to buy a small motorcycle. I’ve been researching safety courses and budgeting for gear, and I think I can handle it responsibly.
MOTHER: I appreciate your research and responsibility, but I'm concerned about the injury risk associated with motorcycles, especially for new riders. Have you considered how much insurance would cost and how you plan to manage noise levels in our neighborhood? Maybe we could postpone the purchase until you've completed a safety course and created a solid budget plan.
FATHER: I support your interest in getting a motorcycle, but we need to set some strong safety rules first. You must complete a mandatory safety course, wear full protective gear at all times, and limit your riding to certain areas until you gain more experience. Additionally, let's set a budget for your purchase and establish savings goals to ensure you're financially prepared. A curfew for riding would also be necessary to keep you safe.
SON: I understand your concerns, and I’m c

"\nTips:\n- Make the dialogue longer by increasing ROUNDS (e.g., 5 or 6).\n- To bias toward a compromise, you can add a final prompt from MOTHER or SON.\n- Adjust tone/length with TEMPERATURE and the role instructions (keep them brief and explicit).\n- If you get a 'model not found' or quota error, switch models or check your billing.\n"